***
***
# 使用Turicreate进行音乐推荐
***
***


In [3]:
import turicreate as tc

# 下载数据
http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt


In [5]:
#train_file = 'http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt'
train_file = '/Users/datalab/bigdata/cjc/millionsong/song_usage_10000.txt'
sf = tc.SFrame.read_csv(train_file, header=False, delimiter='\t', verbose=False)
sf = sf.rename({'X1':'user_id', 'X2':'music_id', 'X3':'rating'})

In [6]:
train_set, test_set = sf.random_split(0.8, seed=1)

In [7]:
popularity_model = tc.popularity_recommender.create(train_set, 
                                                    'user_id', 'music_id', 
                                                    target = 'rating')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 0.970575s

1599753 observations to process; with 10000 unique items.

In [8]:
item_sim_model = gl.item_similarity_recommender.create(train_set, 
                                                       'user_id', 'music_id', 
                                                       target = 'rating', 
                                                       similarity_type='cosine')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 1.03264s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 1.12s                               | 100              | 10000           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.18994s

In [9]:
factorization_machine_model = tc.recommender.factorization_recommender.create(train_set, 
                                                                              'user_id', 'music_id',
                                                                              target='rating')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 1.01785s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 10.5002

Final training RMSE: 3.23894

In [10]:
len(train_set)

1599753

In [11]:
result = tc.recommender.util.compare_models(test_set, 
                                            [popularity_model, item_sim_model, factorization_machine_model],
                                            user_sample=.5, skip_set=train_set)

compare_models: using 34354 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/34354 queries. users per second: 19254.8

recommendations finished on 2000/34354 queries. users per second: 21393.8

recommendations finished on 3000/34354 queries. users per second: 21652.1

recommendations finished on 4000/34354 queries. users per second: 22289.3

...

recommendations finished on 31000/34354 queries. users per second: 16160.2

recommendations finished on 32000/34354 queries. users per second: 16125.3

recommendations finished on 33000/34354 queries. users per second: 16092.7

recommendations finished on 34000/34354 queries. users per second: 16098.5


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    | 0.0002619782266984918  | 6.128571219436259e-05  |
|   2    | 0.00021831518891541178 | 7.390986568034776e-05  |
|   3    | 0.00032019561040926665 | 0.00019722481409035843 |
|   4    | 0.0003420271293008098  | 0.0002710552484110958  |
|   5    | 0.0003958782092332777  | 0.00039069142498776696 |
|   6    | 0.00043177892918825424 | 0.0005468138856453636  |
|   7    | 0.00042831360872928355 | 0.0006389506810876567  |
|   8    | 0.00044390755079466883 | 0.0007327030009271866  |
|   9    | 0.0004851448642564688  | 0.0009366954040388185  |
|   10   | 0.0005006694999126722  | 0.0010614474371010375  |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 7.931771300491865

Per User RMSE (best)
+----------------

In [12]:
K = 10
users = gl.SArray(sf['user_id'].unique().head(100))

In [13]:
recs = item_sim_model.recommend(users=users, k=K)
recs.head()

user_id,music_id,score,rank
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXUQNR12AF72A69D6,3.022422651449839,1
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOUFAZA12AC3DFAB20,1.3368427753448486,2
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOSFSTC12A8C141219,1.091982126235962,3
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOVIWFP12A58A7D1BD,1.045163869857788,4
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOBMTQD12AB01833D0,1.0294516881306965,5
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOCMNRG12AB0189D3F,0.9756437937418619,6
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXOHUM12A67ADC826,0.9506873289744059,7
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOWBFVW12A6D4F612B,0.9092370669047037,8
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXFYTY127E9433E7D,0.8977278073628744,9
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOYBLYP12A58A79D32,0.8970928192138672,10
